In [ ]:
# Import standard libraries for JSON handling and regular expressions
import json
import re
from google.colab import files

In [ ]:
# Upload the CSV file from your computer
try:
    uploaded = files.upload()  # Opens a window to select the file
    if uploaded:
        csv_filename = list(uploaded.keys())[0]  # Get the name of the uploaded file
        print(f"File '{csv_filename}' successfully uploaded!")
    else:
        raise ValueError("No file uploaded. Please upload a CSV file.")
except Exception as e:
    print(f"Error during file upload: {e}")

Saving grafo_estrutura_dbt.ipynb to grafo_estrutura_dbt.ipynb
Saving load_file_to_bigquery.ipynb to load_file_to_bigquery.ipynb
Saving my_dbt_project.ipynb to my_dbt_project.ipynb
Saving read_data_from_bigquey.ipynb to read_data_from_bigquey.ipynb


In [ ]:
# Sensitive keywords to check
keywords = ['token', 'project_id', 'credentials', 'auth', 'key', 'password']

In [ ]:
# Precise regex patterns for tokens
token_patterns = [
    r"AIza[0-9A-Za-z\-_]{35}",            # Google API Key
    r"ya29\.[0-9A-Za-z\-_]+",             # Google OAuth token
    r"gh[pousr]_[0-9A-Za-z]{36,}",        # GitHub tokens (different scopes: p, o, u, s, r)
    r"xox[baprs]-[0-9A-Za-z-]+",          # Slack tokens (bot, user, etc.)
    r"(?i)(token|key|auth)[\"'\s:=]+[\"']?([A-Za-z0-9_\-]{20,})[\"']?"  # Generic pattern with common token labels
]

In [ ]:
for filename in uploaded.keys():
    print(f"\nChecking file: {filename}")

    try:
        with open(filename, 'r', encoding='utf-8') as f:
            content = f.read()

        notebook = json.loads(content)

        # Check each cell
        for i, cell in enumerate(notebook.get("cells", [])):
            if cell.get("cell_type") != "code":
                continue
            code_lines = cell.get("source", [])
            code = ''.join(code_lines)

            # Check for suspicious keywords
            for keyword in keywords:
                for line in code_lines:
                    if re.search(rf"{keyword}\s*=\s*[\"']?.+?[\"']?", line, re.IGNORECASE):
                        print(f"[File: {filename} | Cell: {i}] Suspicious keyword: `{keyword}` → {line.strip()}")

            # Check for token patterns
            for pattern in token_patterns:
                for line in code_lines:
                    matches = re.findall(pattern, line)
                    if matches:
                        for match in matches:
                            # If the regex captured groups, extract the token only
                            if isinstance(match, tuple):
                                match = match[1]
                            print(f"[File: {filename} | Cell: {i}] Possible token found: {match} → {line.strip()}")

        # Check for project_id (anywhere in the file)
        project_ids = re.findall(r'project[-_]id["\']?\s*[:=]\s*["\']([\w-]+)', content, re.IGNORECASE)
        for project_id in project_ids:
            print(f"[File: {filename}] Detected project_id: {project_id}")

    except Exception as e:
        print(f"Error while processing {filename}: {e}")


📁 Checking file: grafo_estrutura_dbt.ipynb

📁 Checking file: load_file_to_bigquery.ipynb
⚠️  [File: load_file_to_bigquery.ipynb | Cell: 3] Suspicious keyword: `project_id` → PROJECT_ID = "meu-projeto-dbt"  # Substitua pelo seu project_id

📁 Checking file: my_dbt_project.ipynb
⚠️  [File: my_dbt_project.ipynb | Cell: 21] Suspicious keyword: `project_id` → project_id = "meu-projeto-dbt"

📁 Checking file: read_data_from_bigquey.ipynb
⚠️  [File: read_data_from_bigquey.ipynb | Cell: 3] Suspicious keyword: `project_id` → project_id = 'meu-projeto-dbt'  # substitua com seu ID do projeto
🆔 [File: read_data_from_bigquey.ipynb] Detected project_id: meu-projeto-dbt
